In [232]:
import os
import pickle
import json
import numpy as np
import importlib

from tqdm import tqdm

## GPU kNN

In [219]:
import numpy as np
import faiss
import sys

import knn_gpu
importlib.reload(knn_gpu)

def run_knn_features(feature_vectors, test_vectors=None, k=5, flat=True,
                     verbose=False, dist=False, gpu=False):
    nb, d = feature_vectors.shape
    if type(test_vectors) is not np.ndarray:
        if flat:
            D, I = knn_gpu.knn_flat(feature_vectors, feature_vectors, d, k, verbose, gpu)
        else:
            D, I = knn_gpu.knn_ivf(feature_vectors, feature_vectors, d, k, verbose, gpu)
    else:
        if flat:
            D, I = knn_gpu.knn_flat(feature_vectors, test_vectors, d, k, verbose, gpu)
        else:
            D, I = knn_gpu.knn_ivf(feature_vectors, test_vectors, d, k, verbose, gpu)

    if dist:
        return D, I 
    else:
        return I

## CPU kNN

In [45]:
# imports
import os
import time
import numpy as np
import pickle
import h5py
import faiss

class knn_query():
    """
    Class for kNN query using FAISS
    """

    def __init__(self, dims, xb, labels):
        """ Constructor
            Args:
                dims = number of dimensions
                index = FAISS index object
                xb = numpy array of feature vectors
                labels = dict, index to frame path mapping
                isTrained = boolean, is the index trained?
        """
        self.dims = dims
        self.index = None
        self.xb = xb
        self.labels = labels
        self.isTrained = False

    def train_knn_index(self):
        """
        This function trains the index using the extracted features so that
        kNN query can be performed. It just needs to be run with the extracted
        features.
        """
        # build the index
        s_time = time.time()
        self.index = faiss.IndexFlatL2(self.dims)
        self.isTrained = True
        # add feature vectors to the index
        print('Adding feature vectors to the index...')
        self.index.add(self.xb)
        e_time = time.time()
        index_time = round(e_time - s_time, 3)
        print('{} feature vectors added'.format(self.index.ntotal))
        print('Index finished in {}s'.format(index_time))

    def perform_query(self, xq, k):
        """
        This method performs the kNN query on trained index of the
        FAISS index object.
        """
        if not self.isTrained:
            print('You must train the index first!')
        else:
            # perform the query
            print('Performing {} nearest neighbor search...'.format(k))
            s_time = time.time()
            D, I = self.index.search(x=xq, k=k)
            e_time = time.time()
            query_time = round(e_time - s_time, 3)
            return D, I, query_time


def output_results(I, labels, query_image, query_time):
    print('\nQuery Completed in {}s'.format(query_time))
    print('\nQuery Frame:\n', query_image)
    print('\n')
    print('Top Similar Frames:')
    for i, ind in enumerate(I[0]):
        print(i+1, ': ', labels[ind])

## Convert Extracted Features File

In [ ]:
# extracted features files path
extracted_features_path = '/mnt/e/3dcnn_features/'
extracted_features_filename = 'ucf101_features'

In [ ]:
# load extracted features
file = open(os.path.join(extracted_features_path, extracted_features_filename),'rb')
features = pickle.load(file)

In [233]:
with open('output.json', 'r') as f:
    features = json.load(f)

In [236]:
feature_vectors = []
ind2video_mapping = {}
video2ind_mapping = {}
for i, video in enumerate(features):
    feature_vectors.append(video['clips'][0]['features'])
    ind2video_mapping[i] = video['video']
    video2ind_mapping[video['video']] = i
    
feature_vectors = np.array(feature_vectors)
feature_vectors = feature_vectors.astype('float32')

In [241]:
ind2video_mapping

{0: 'v_ApplyEyeMakeup_g02_c03.avi',
 1: 'v_Kayaking_g17_c06.avi',
 2: 'v_ApplyEyeMakeup_g04_c02.avi',
 3: 'v_LongJump_g11_c02.avi'}

## Test kNN on Features

In [242]:
video2ind_mapping

{'v_ApplyEyeMakeup_g02_c03.avi': 0,
 'v_Kayaking_g17_c06.avi': 1,
 'v_ApplyEyeMakeup_g04_c02.avi': 2,
 'v_LongJump_g11_c02.avi': 3}

In [245]:
# test CPU knn
d = 512
k = 3
query = knn_query(d, feature_vectors, ind2video_mapping)
query.train_knn_index()

Adding feature vectors to the index...
4 feature vectors added
Index finished in 0.0s


In [246]:
# test input 
n_videos = len(features)
# np.random.seed(0)
n = np.random.randint(0, n_videos)
print(n)

# get query vector
query_features = np.reshape(feature_vectors[n, :].astype(np.float32), (1, -1))
print(ind2video_mapping[n])
print(query_features.shape)

1
v_Kayaking_g17_c06.avi
(1, 512)


In [247]:
# CPU kNN
D, I, q_time = query.perform_query(query_features, k)
output_results(I, ind2video_mapping, ind2video_mapping[n], q_time)

Performing 3 nearest neighbor search...

Query Completed in 0.0s

Query Frame:
 v_Kayaking_g17_c06.avi


Top Similar Frames:
1 :  v_Kayaking_g17_c06.avi
2 :  v_ApplyEyeMakeup_g04_c02.avi
3 :  v_ApplyEyeMakeup_g02_c03.avi


In [355]:
# GPU kNN
q_time_gpu = None
I_gpu = run_knn_features(feature_vectors=feature_vectors, test_vectors=query_features, gpu=False)
output_results(I_gpu, ind2video_mapping, ind2video_mapping[n], q_time_gpu)

KeyError: 10

## Function Tests

In [335]:
def similar_3dcnn_ucf_video(video_path, feature_vectors, k=5, dist=False, verbose=False):
    """
    This function extracts features from the query video and performs kNN similarity search.
    """
    try:
#         query_features = extract_features_from_vid(video_path)
        distances, feature_indices = run_knn_features(feature_vectors, test_vectors=query_features,
                                                    k=k, dist=True)
        print(distances)
        if verbose:
            print(color_labels[feature_indices][0])
        if dist:
            return list(distances[0]), list(feature_vectors[feature_indices][0])
        else:
            return list(feature_vectors[feature_indices][0])
    except:
        print('No video found!')

In [336]:
cnn3d_dist, cnn3d_indices = similar_3dcnn_ucf_video(True, feature_vectors, k=3, dist=True, verbose=False)

[[  0.      321.06046 376.89996]]


In [342]:
print(cnn3d_dist)
print(len(cnn3d_indices))

[0.0, 321.06046, 376.89996]
3
